In [1]:
import tkinter as tk
import pygame 
import time
import datetime
import random
from mutagen.mp3 import MP3
import pandas as pd
import serial

pygame 2.5.2 (SDL 2.28.3, Python 3.11.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class BiosemiTrigger(serial.Serial):
    def __init__(self, Serial_Port, initial_delay = 3):
        super().__init__(Serial_Port, baudrate = 115200)
        time.sleep(initial_delay)

    def send_trigger(self, root, duration = 8, signal_byte = 0b00000010):
        if not (0 <= signal_byte <= 255):
                raise ValueError("signal_byte must be between 0 and 255")
        self.write(bytes([signal_byte]))
        #time.sleep(0.001) #1ms pulse duration
        root.after(duration, lambda: self.write(bytes([0])))

In [3]:
def create_canvas(tk_window):
    screen_width = tk_window.winfo_screenwidth()
    screen_height = tk_window.winfo_screenheight()

    canvas = tk.Canvas(tk_window, width = screen_width, height = screen_height, bg = "black", highlightthickness = 0)
    canvas.pack(fill = "both", expand = True)
    return canvas

In [4]:
def save(input):
    df = pd.read_csv(f"{input}.csv")
    if not df.empty: 
      timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
      begin_df = df.tail(1)
      begin_df.reset_index(drop = True, inplace = True)
      end_df = pd.DataFrame({"End": [f"{timestamp}"]})
      comb_df = begin_df.combine_first(end_df)
      comb_df = comb_df[["State", "Begin", "End"]]
      df.drop(df.tail(1).index, inplace = True)
      df = pd.concat([df, comb_df], ignore_index = True)
      del begin_df, end_df, comb_df
    df.to_csv(f"{input}.csv", index = False)  

In [5]:
def create_cross(canvas, tk_window, input):
     
    save(input)

    df = pd.read_csv(f"{input}.csv")
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    temp_df = pd.DataFrame({"State": "Resting State", "Begin": [f"{timestamp}"], "End" :""})
    df = pd.concat([df, temp_df], ignore_index = True)
    df.to_csv(f"{input}.csv", index = False)
    del temp_df

    screen_width = tk_window.winfo_screenwidth()
    screen_height = tk_window.winfo_screenheight()
    
    cross_length = 250
    line_thickness = 13
    x_center = screen_width // 2
    y_center = screen_height // 2

    horizontal_line = canvas.create_line(x_center - cross_length // 2, y_center,
                    x_center + cross_length // 2, y_center,
                    fill = "white", width = line_thickness)

    vertical_line = canvas.create_line(x_center, y_center - cross_length // 2,
                    x_center, y_center + cross_length // 2,
                    fill = "white", width = line_thickness)
    

In [6]:
def stimuli_duration():  
    #audio = MP3("Gotcha bitch.mp3")
    length = 1000#round(audio.info.length * 1000 + 100)
    return length 

In [7]:
def stimuli(tk_window, biosemi_trigger):
    #pygame.mixer.music.load("Gotcha bitch.mp3")
    length = stimuli_duration()
    biosemi_trigger.send_trigger(tk_window, length)
    #pygame.mixer.music.play()

In [8]:
def stim_bg(canvas, input, tk_window, biosemi_trigger):
    save(input)
   
    df = pd.read_csv(f"{input}.csv")
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") 
        
    temp_df = pd.DataFrame({"State": "Stimuli", "Begin": [f"{timestamp}"], "End" :""})
    df = pd.concat([df, temp_df], ignore_index = True)
    df.to_csv(f"{input}.csv", index = False)
    del temp_df
    
    canvas.delete('all')
    stimuli(tk_window, biosemi_trigger)

In [9]:
def random_time():
    randomtime = int(round(random.uniform(5.0, 8.5), 1) * 1000)
    return randomtime

In [10]:
def stimuli_cicle(time, tk_window, canvas, input, biosemi_trigger):
    length = stimuli_duration()
    wait = random_time()

    tk_window.after(time+wait, lambda: stim_bg(canvas, input, tk_window, biosemi_trigger))
    tk_window.after(time+wait+length, lambda: create_cross(canvas, tk_window, input))
    return wait

In [11]:
def start_exp(input, biosemi_trigger): 
    df = pd.DataFrame()
    
    for col in ["State", "Begin", "End"]:
        df[col] = ""

    df.to_csv(f"{input}.csv", index = False)
    
    w2 = tk.Tk()
    w2.title("Experiment")
    w2.attributes("-fullscreen", True)

    canvas = create_canvas(w2)
    create_cross(canvas, w2, input)
    
    time = 0
    for i in range(3):
        wait = stimuli_cicle(time, w2, canvas, input, biosemi_trigger)
        time = wait + time
        
    wait_close = time + stimuli_duration() + 2000
    w2.after(wait_close, lambda: save(input))
    w2.after(wait_close, lambda: w2.destroy())

In [13]:
biosemi_trigger = BiosemiTrigger("COM4", initial_delay = 1)

w1 = tk.Tk()
w1.geometry("300x200")
w1.title("Start")
pygame.mixer.init()


frame = tk.Frame(w1)
frame.pack(expand = True)

entry = tk.Entry(frame)
entry.pack(pady = (0, 5))

b = tk.Button(frame, text = "Start", command = lambda: start_exp(entry.get(), biosemi_trigger))
b.pack()

w1.mainloop()
biosemi_trigger.close()
pygame.quit()